In [1]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# API Endpoint
NEO_FEED_URL = "https://api.nasa.gov/neo/rest/v1/feed"

def fetch_neows_data(api_key, start_date=None, end_date=None):
    if start_date is None:
        start_date = (datetime.today() - timedelta(days=7)).strftime("%Y-%m-%d")
    if end_date is None:
        end_date = datetime.today().strftime("%Y-%m-%d")

    response = requests.get(NEO_FEED_URL, params={
        'start_date': start_date,
        'end_date': end_date,
        'api_key': api_key
    })

    data = response.json()
    asteroids = []
    for day_data in data['near_earth_objects'].values():
        for asteroid in day_data:
            asteroids.append({
                'id': asteroid['id'],
                'name': asteroid['name'],
                'close_approach_date': asteroid['close_approach_data'][0]['close_approach_date'],
                'miss_distance_km': float(asteroid['close_approach_data'][0]['miss_distance']['kilometers']),
                'absolute_magnitude_h': asteroid['absolute_magnitude_h'],
                'estimated_diameter_km_min': asteroid['estimated_diameter']['kilometers']['estimated_diameter_min'],
                'estimated_diameter_km_max': asteroid['estimated_diameter']['kilometers']['estimated_diameter_max'],
                'is_potentially_hazardous': asteroid['is_potentially_hazardous_asteroid']
            })

    return pd.DataFrame(asteroids)

# Usage example
if __name__ == "__main__":
    api_key = "UFvGKAnwQkIgSwLrTndbScEG3Z4J4DeypSJiC5FJ" 
    neows_data = fetch_neows_data(api_key)
    neows_data.to_csv('neows_data.csv', index=False)
